In [ ]:
# ======================================================
# 🤖 Telegram Q&A Extractor for Colab (v2)
# ======================================================

# ⬇️ Устанавливаем зависимости
!pip install pandas ipywidgets

import json
import pandas as pd
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets

# ======================================================
# 🔧 Настройки ключевых слов
# ======================================================
KEYWORDS = [
    "подскажите", "вопрос", "#вопрос", "помощь", "помогите"
]

# ======================================================
# 📂 Загрузка файла
# ======================================================
print("📤 Загрузите JSON-дамп канала Telegram (экспорт из Telegram Desktop).")
uploaded = files.upload()

# Берём имя загруженного файла
input_file = list(uploaded.keys())[0]
print(f"✅ Файл загружен: {input_file}")

# ======================================================
# 📄 Извлечение сообщений с вопросами
# ======================================================
def message_contains_keywords(text: str, keywords: list) -> bool:
    """Проверяет, содержит ли сообщение одно из ключевых слов."""
    text_lower = text.lower()
    return any(keyword.lower() in text_lower for keyword in keywords)

def extract_questions(input_file: str):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    messages = data.get("messages", data)
    rows = []

    for msg in messages:
        if isinstance(msg, dict):
            text = msg.get("text", "")
            if isinstance(text, list):
                # Telegram JSON часто хранит текст как список фрагментов
                text = "".join([t if isinstance(t, str) else t.get("text", "") for t in text])

            if message_contains_keywords(text, KEYWORDS):
                name = msg.get("from", "—")
                username = msg.get("from_id", "—")
                date = msg.get("date", "—")

                rows.append({
                    "Дата": date,
                    "Имя": name,
                    "Тег / ID": username,
                    "Вопрос": text.strip()
                })

    df = pd.DataFrame(rows)
    if not df.empty:
        # Преобразуем дату к читаемому виду, если возможно
        try:
            df["Дата"] = pd.to_datetime(df["Дата"])
        except Exception:
            pass
    return df

df = extract_questions(input_file)
print(f"🔍 Найдено {len(df)} сообщений, содержащих вопросы.")

# ======================================================
# 📊 Интерактивная таблица с фильтрацией
# ======================================================
if not df.empty:
    name_filter = widgets.Text(description='Имя:')
    tag_filter = widgets.Text(description='Тег:')
    question_filter = widgets.Text(description='Вопрос:')
    date_filter = widgets.Text(description='Дата (YYYY-MM-DD):')

    def filter_table(change=None):
        filtered = df[
            df["Имя"].str.contains(name_filter.value, case=False, na=False) &
            df["Тег / ID"].astype(str).str.contains(tag_filter.value, case=False, na=False) &
            df["Вопрос"].str.contains(question_filter.value, case=False, na=False) &
            df["Дата"].astype(str).str.contains(date_filter.value, case=False, na=False)
        ]
        clear_output(wait=True)
        display(ui, filtered)

    ui = widgets.VBox([date_filter, name_filter, tag_filter, question_filter])
    out = widgets.interactive_output(filter_table, {
        'change': widgets.fixed(None)
    })

    display(ui, df)
    out
else:
    print("⚠️ Сообщения с вопросами не найдены.")


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)